# 1. Import Libraries:

In [1]:
from matplotlib import pyplot as plt
import mediapipe as mp
import numpy as np
import time
import cv2
import os
from utils import *

In [2]:
text_background = (198, 63, 88)  # PURPLE
corner_color = (53, 53, 249)  # RED
text_color = (239, 239, 239)  # WHITE
border_color = (61, 147, 8)  # GREEN

# 5. Setup Folders for Collection:

In [3]:
DATA_PATH = os.path.join("Data")
actions = np.array(["Hello", "Nothing"])
no_sequences = 5
sequence_length = 5

In [76]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 6. Collect MP Keypoints:

In [4]:
mp_holistic = mp.solutions.holistic # type: ignore
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)
no_frames_counter = 0
no_sequences_counter = 0
current_action = "Nothing" # Done Actions: Hello, Nothing

while True:
    _, image = cap.read()
    # image = cv2.flip(image, 1)
    
    image, results = mediapipe_detections(image, holistic)
    draw_landmarks(image, results, corner_color)
    right_hand, left_hand = extract_keypoints(results)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("s"):
        npy_path = os.path.join(DATA_PATH, current_action, str(no_sequences_counter), f"{no_frames_counter}.npy")
        np.save(npy_path, right_hand)
        no_frames_counter += 1
        if no_frames_counter == sequence_length:
            no_frames_counter = 0
            no_sequences_counter += 1
            if no_sequences_counter == no_sequences:
                break
    
    cv2.putText(image, f"Collecting Frames for '{current_action}'", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Video Num: {no_sequences_counter}", (15, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Frame Num: {no_frames_counter}", (15, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("Image", image)
    
    if key == ord("n"):
        cv2.imwrite(f"image-{no_frames_counter}-{no_sequences_counter}.jpg", image)
    
    if key == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()


C:\Users\Choaib ELMADI\Downloads\D.I.F.Y\Electronics\Robotics\6- EchoLens\Sign Language Model (DLM)\SignLangVenvDLM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# 7. Preprocess Data and Create Labels and Features:

In [78]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [79]:
label_map = {label:num for num, label in enumerate(actions)}

In [80]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [81]:
X = np.array(sequences)
print(X.shape)

(10, 5, 63)


In [82]:
y = to_categorical(labels).astype(int)

In [83]:
y

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 8. Build and Train LSTM Neural Network:

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [7]:
log_dir = os.path.join("logs")
tb_callback = TensorBoard(log_dir=log_dir) # for moritoring the NN training

In [9]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(5, 63)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(2, activation="softmax"))

In [88]:
res = [.7, .3]
actions[np.argmax(res)]
actions.shape[0]

2

In [89]:
X_train.shape
y_train.shape

(9, 2)

In [10]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [91]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - categorical_accuracy: 0.4444 - loss: 0.6946
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - categorical_accuracy: 0.4444 - loss: 0.6935
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - categorical_accuracy: 0.5556 - loss: 0.6928
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - categorical_accuracy: 0.5556 - loss: 0.6922
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - categorical_accuracy: 0.5556 - loss: 0.6917
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - categorical_accuracy: 0.5556 - loss: 0.6914
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - categorical_accuracy: 0.5556 - loss: 0.6909
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 0.5556 - loss: 0.6904
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - categorical_accuracy: 0.5556 - loss: 0.6900
Epoch 10/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - categorical_accuracy: 0.5556 - loss: 0.6894
Epoch 11/1000
1/1 ━━━━━━━━━━━━

Epoch 63/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - categorical_accuracy: 0.7778 - loss: 0.2302
Epoch 64/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - categorical_accuracy: 0.7778 - loss: 0.2224
Epoch 65/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - categorical_accuracy: 0.8889 - loss: 0.1817
Epoch 66/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - categorical_accuracy: 1.0000 - loss: 0.1310
Epoch 67/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - categorical_accuracy: 1.0000 - loss: 0.0886
Epoch 68/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - categorical_accuracy: 1.0000 - loss: 0.0577
Epoch 69/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - categorical_accuracy: 1.0000 - loss: 0.0397
Epoch 70/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - categorical_accuracy: 1.0000 - loss: 0.0303
Epoch 71/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 1.0000 - loss: 0.0258
Epoch 72/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - categorical_accuracy: 1.0000 - loss: 0.0238
Epoch 73/1000
1/1 ━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - categorical_accuracy: 1.0000 - loss: 0.0017
Epoch 125/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - categorical_accuracy: 1.0000 - loss: 0.0016
Epoch 126/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - categorical_accuracy: 1.0000 - loss: 0.0015
Epoch 127/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 1.0000 - loss: 0.0014
Epoch 128/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 1.0000 - loss: 0.0013
Epoch 129/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 1.0000 - loss: 0.0012
Epoch 130/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - categorical_accuracy: 1.0000 - loss: 0.0011
Epoch 131/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - categorical_accuracy: 1.0000 - loss: 0.0011
Epoch 132/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - categorical_accuracy: 1.0000 - loss: 9.9531e-04
Epoch 133/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 1.0000 - loss: 9.3299e-04
Epoch 134/1000
1/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - categorical_accuracy: 1.0000 - loss: 6.2153e-05
Epoch 185/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - categorical_accuracy: 1.0000 - loss: 5.9637e-05
Epoch 186/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - categorical_accuracy: 1.0000 - loss: 5.7346e-05
Epoch 187/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - categorical_accuracy: 1.0000 - loss: 5.5227e-05
Epoch 188/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - categorical_accuracy: 1.0000 - loss: 5.3228e-05
Epoch 189/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - categorical_accuracy: 1.0000 - loss: 5.1294e-05
Epoch 190/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - categorical_accuracy: 1.0000 - loss: 4.9467e-05
Epoch 191/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - categorical_accuracy: 1.0000 - loss: 4.7732e-05
Epoch 192/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - categorical_accuracy: 1.0000 - loss: 4.6063e-05
Epoch 193/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - categorical_accuracy: 1.0000 - lo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - categorical_accuracy: 1.0000 - loss: 1.3748e-05
Epoch 244/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - categorical_accuracy: 1.0000 - loss: 1.3457e-05
Epoch 245/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - categorical_accuracy: 1.0000 - loss: 1.3219e-05
Epoch 246/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - categorical_accuracy: 1.0000 - loss: 1.2980e-05
Epoch 247/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - categorical_accuracy: 1.0000 - loss: 1.2729e-05
Epoch 248/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - categorical_accuracy: 1.0000 - loss: 1.2530e-05
Epoch 249/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 1.0000 - loss: 1.2318e-05
Epoch 250/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - categorical_accuracy: 1.0000 - loss: 1.2080e-05
Epoch 251/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - categorical_accuracy: 1.0000 - loss: 1.1841e-05
Epoch 252/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - categorical_accuracy: 1.0000 

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 716/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 717/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 718/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 719/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 720/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 721/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 722/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 723/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - categorical_accuracy: 1.0000 - loss: 1.9868e-07
Epoch 724/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - categorical_accuracy: 1.0000 - 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - categorical_accuracy: 1.0000 - loss: 1.5895e-07
Epoch 775/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 1.0000 - loss: 1.5895e-07
Epoch 776/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 1.0000 - loss: 1.5895e-07
Epoch 777/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 1.0000 - loss: 1.5895e-07
Epoch 778/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - categorical_accuracy: 1.0000 - loss: 1.5895e-07
Epoch 779/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - categorical_accuracy: 1.0000 - loss: 1.5895e-07
Epoch 780/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - categorical_accuracy: 1.0000 - loss: 1.5895e-07
Epoch 781/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - categorical_accuracy: 1.0000 - loss: 1.4570e-07
Epoch 782/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - categorical_accuracy: 1.0000 - loss: 1.4570e-07
Epoch 783/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - categorical_accuracy: 1.0000 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 834/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 835/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 836/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 837/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 838/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 839/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 840/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 841/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - categorical_accuracy: 1.0000 - loss: 1.3245e-07
Epoch 842/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 1.0000 - l

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 893/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 894/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 895/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 896/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 897/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 898/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 899/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 900/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - categorical_accuracy: 1.0000 - loss: 1.1921e-07
Epoch 901/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - categorical_accuracy: 1.0000 -

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 952/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 953/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 954/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 955/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 956/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 957/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 958/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 959/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 1.0000 - loss: 9.2718e-08
Epoch 960/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - categorical_accuracy: 1.0000 - 

In [12]:
model.load_weights("action.keras")

# 9. Make Predictions:

In [97]:
model.summary(line_length=100)

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                               ┃ Output Shape                    ┃           Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lstm_40 (LSTM)                             │ (None, 5, 64)                   │            32,768 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ lstm_41 (LSTM)                             │ (None, 5, 128)                  │            98,816 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ lstm_42 (LSTM)                             │ (None, 64)                      │            49,408 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_39 (Dense)                           │ (None, 64)                      │             4,160 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_40 (Dense)                           │ (None, 32)                      │             2,080 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_41 (Dense)                           │ (None, 2)                       │                66 │
└────────────────────────────────────────────┴─────────────────────────────────┴───────────────────┘

 Total params: 561,896 (2.14 MB)

 Trainable params: 187,298 (731.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 374,598 (1.43 MB)

In [109]:
results = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [116]:
actions[np.argmax(results[0])]

'Nothing'

In [115]:
actions[np.argmax(y_test[0])]

'Nothing'

# 10. Save Weights:

In [118]:
model.save("action.keras")

# 11. Evaluation using Confusion Matrix and Accuracy:

In [121]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [122]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [123]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [126]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[0, 0],
        [0, 1]]], dtype=int64)

In [128]:
accuracy_score(ytrue, yhat)

1.0

In [19]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# 12. Test Real Time:

In [31]:
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    # frame = cv2.flip(frame, 1)

    frame, results = mediapipe_detections(frame, holistic)
    draw_landmarks(frame, results, corner_color)

    right_hand, _ = extract_keypoints(results)
    sequence.append(right_hand)
    sequence = sequence[-5:]
    
    if len(sequence) == 5:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        predictions.append(np.argmax(res))
    
        if np.unique(predictions[-10:])[0] == np.argmax(res):
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                        print(" ".join(sentence))
                else:
                    sentence.append(actions[np.argmax(res)])
            
    frame = prob_viz(res, actions, frame, colors)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Nothing Hello Nothing Hello Nothing Hello Nothing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
